In [ ]:
## IMPORTS


import os
import black
from openai import OpenAI
from google.colab import userdata
import gradio as gr

In [ ]:
!pip install black

In [7]:
## API ACCESS

OPENAI_API_KEY = userdata.get("OPENAI_API_KEY")
ANTHROPIC_API_KEY = userdata.get("ANTHROPIC_API_KEY")

if not OPENAI_API_KEY:
    raise ValueError("OPENAI_API_KEY not found in Colab Secrets.")

if not ANTHROPIC_API_KEY:
    raise ValueError("ANTHROPIC_API_KEY not found in Colab Secrets.")

print("API Keys Loaded Successfully")

✅ API Keys Loaded Successfully


In [8]:
## CLIENT INITILIZATION

openai_client = OpenAI(api_key=OPENAI_API_KEY)

anthropic_client = OpenAI(
    api_key=ANTHROPIC_API_KEY,
    base_url="https://api.anthropic.com/v1"
)

print("Clients initialized successfully")

✅ Clients initialized successfully


In [ ]:
## MODEL CONFIGURATION

OPENAI_HIGH = "gpt-4o"
OPENAI_LOW = "gpt-4o-mini"

CLAUDE_HIGH = "claude-3-opus-20240229"
CLAUDE_LOW = "claude-3-haiku-20240307"

In [10]:
## CLIENT-MODEL SWITCHING

def get_client_and_model(provider="openai", tier="low"):

    if provider == "openai":
        model = OPENAI_HIGH if tier == "high" else OPENAI_LOW
        return openai_client, model

    elif provider == "claude":
        model = CLAUDE_HIGH if tier == "high" else CLAUDE_LOW
        return anthropic_client, model

    else:
        raise ValueError("Unsupported provider.")

In [ ]:
## SYSTEM PROMPT

def build_system_prompt(include_tests=False):

    base_prompt = """
You are a senior Python engineer.

Your task:

1. Add professional docstrings.
2. Add meaningful inline comments.
3. Do NOT change logic.
4. Do NOT wrap in markdown.
5. Return ONLY raw executable Python code.

Docstrings must include:
- Description
- Args
- Returns
"""

    if include_tests:
        base_prompt += """

6. After the updated code, generate pytest unit tests.
   - Cover normal and edge cases.
   - Use professional structure.
"""

    return base_prompt

In [ ]:
## PEP 8 FORMATTER

def format_code_pep8(code):
    try:
        return black.format_str(code, mode=black.FileMode())
    except Exception:
        return code

In [ ]:
## DOCSTRING GENERATOR

def generate_docstrings(code, provider="openai", tier="low", include_tests=False):

    client, model = get_client_and_model(provider, tier)
    system_prompt = build_system_prompt(include_tests)

    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": code}
        ],
        temperature=0
    )

    content = response.choices[0].message.content
    content = format_code_pep8(content)

    usage = response.usage
    total_tokens = usage.total_tokens

    cost_per_1k_tokens = 0.005
    estimated_cost = (total_tokens / 1000) * cost_per_1k_tokens

    metadata = f"""
Prompt Tokens: {usage.prompt_tokens}
Completion Tokens: {usage.completion_tokens}
Total Tokens: {usage.total_tokens}
Estimated Cost: ${estimated_cost:.6f}
"""

    return content, metadata

In [29]:
## STREAM DOCSTRING GENERATOR

def generate_docstrings_stream(code, provider="openai", tier="low", include_tests=False):

    client, model = get_client_and_model(provider, tier)
    system_prompt = build_system_prompt(include_tests)

    stream = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": code}
        ],
        temperature=0,
        stream=True
    )

    full_response = ""

    for chunk in stream:
        if chunk.choices[0].delta.content:
            content = chunk.choices[0].delta.content
            full_response += content
            yield full_response, ""   # live stream

    #POST-PROCESS AFTER STREAM ENDS

    formatted = format_code_pep8(full_response)

    # Get usage from final chunk
    final_response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": code}
        ],
        temperature=0
    )

    usage = final_response.usage
    total_tokens = usage.total_tokens
    cost_per_1k_tokens = 0.005
    estimated_cost = (total_tokens / 1000) * cost_per_1k_tokens

    metadata = f"""
Prompt Tokens: {usage.prompt_tokens}
Completion Tokens: {usage.completion_tokens}
Total Tokens: {usage.total_tokens}
Estimated Cost: ${estimated_cost:.6f}
"""

    yield formatted, metadata

In [39]:
## FILE UPLOAD SUPPORT

def load_file(file):
    if file is None:
        return ""
    return file.read().decode("utf-8")

In [40]:
## COPY TO CLIPBOARD support

def copy_output(code):
    return code

In [ ]:
## TOKEN USAGE COST CALCULATOR

def generate_docstrings(code, provider="openai", tier="low", include_tests=False):

    client, model = get_client_and_model(provider, tier)
    system_prompt = build_system_prompt(include_tests)

    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": code}
        ],
        temperature=0
    )

    content = response.choices[0].message.content

    # Token usage
    usage = response.usage
    prompt_tokens = usage.prompt_tokens
    completion_tokens = usage.completion_tokens
    total_tokens = usage.total_tokens

    # Rough cost estimate (adjust per model if needed)
    cost_per_1k_tokens = 0.005  # Example estimate
    estimated_cost = (total_tokens / 1000) * cost_per_1k_tokens

    metadata = f"\n\n# --- METADATA ---\n# Prompt Tokens: {prompt_tokens}\n# Completion Tokens: {completion_tokens}\n# Total Tokens: {total_tokens}\n# Estimated Cost: ${estimated_cost:.6f}\n"

    return content + metadata

In [ ]:
## GRADIO INTERFACE

def docstring_app(code, provider, tier, streaming, include_tests):

    if not code or not code.strip():
        yield "#Please paste valid Python code.", ""
        return

    if streaming:
        yield from generate_docstrings_stream(
            code, provider, tier, include_tests
        )
    else:
        content, metadata = generate_docstrings(
            code, provider, tier, include_tests
        )
        yield content, metadata

def clear_fields():
    return "", ""


with gr.Blocks() as demo:

    gr.Markdown("## AI Docstring & Unit Test Generator")
    gr.Markdown("Production-ready multi-model AI code enhancement tool.")

    with gr.Row():
        provider = gr.Dropdown(
            choices=["openai", "claude"],
            value="openai",
            label="Provider"
        )
        tier = gr.Dropdown(
            choices=["low", "high"],
            value="low",
            label="Model Tier"
        )

    streaming = gr.Checkbox(label="Enable Streaming", value=True)
    include_tests = gr.Checkbox(label="Generate Unit Tests", value=False)

    file_upload = gr.File(label="Upload Python File (.py)", file_types=[".py"])

    code_input = gr.Code(label="Paste Python Code", language="python")

    file_upload.change(load_file, inputs=file_upload, outputs=code_input)

    output = gr.Code(label="Generated Code", language="python")
    metadata_output = gr.Textbox(label="Token Usage & Cost", lines=4)

    with gr.Row():
        generate_btn = gr.Button("Generate")
        clear_btn = gr.Button("Clear", variant="secondary")

    generate_btn.click(
        docstring_app,
        inputs=[code_input, provider, tier, streaming, include_tests],
        outputs=[output, metadata_output]
    )

    clear_btn.click(
        clear_fields,
        outputs=[code_input, output, metadata_output]
    )

demo.launch(debug=True)

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://e1dd5ae17d58d5778e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
